In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
import pickle
import gc
import ctypes
from pathlib import Path
import logging
import json
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor
import datetime
from typing import Tuple, Optional
import warnings
import fnmatch
warnings.filterwarnings('ignore')

import cuml
import cudf
from cuml.linear_model import LogisticRegression as cuCML_LogisticRegression
import cupy as cp
from sklearn.multioutput import MultiOutputClassifier
from cuml.model_selection import train_test_split
# Optional: for evaluation
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from cuml.preprocessing import StandardScaler

In [2]:
print(f"Polars version: {pl.__version__}")
print(f"CuPy version: {cp.__version__}")

try:
    #import cuml
    print(f"cuML version: {cuml.__version__}")
    print(f"cuML version imported successfully.")
    
except ImportError as e:
    print(f"cuML could not be imported. Ensure RAPIDS is installed correctly. Error: {e}")
    # If Cuml cannot be imported, the rest of the notebook will not work.
    # In this case, it may make sense to stop Execution.
    raise

Polars version: 1.25.0
CuPy version: 13.6.0
cuML version: 25.02.01
cuML version imported successfully.


In [3]:
try:
    gpu_array = cp.arange(10)
    print(f"CuPy array on this device: {gpu_array.device}")
except cp.cuda.runtime.CUDARuntimeError as e:
    print(f"CuPy device can't be started: {e}")
    print("Make sure your CUDA drivers and cuPy installation are correct.")

CuPy array on this device: <CUDA Device 0>


## Setup the test data on which predictions need to be generated

In [4]:
df_test = pl.read_parquet('/kaggle/input/cafa6-protein-go-terms-feat-labels/test_protein_features_esm2_480.parquet')
print("Shape of test features", df_test.shape)
print(df_test.head(5))

prots_for_submission = np.array(pl.Series(df_test.select(pl.col('protein_accession_id'))).to_list())
print("prots_for_submission -- ", prots_for_submission.shape)

submission_embed_np_array = df_test['embedding_arrays'].to_numpy().astype(np.float32)
print(f"NumPy array shape: {submission_embed_np_array.shape}")
print(f"NumPy array dtype: {submission_embed_np_array.dtype}")

submission_embeds_cp_array = cp.array(submission_embed_np_array)
print(f"CuPy array shape: {submission_embeds_cp_array.shape}")
print(f"CuPy array dtype: {submission_embeds_cp_array.dtype}")
print(f"CuPy array device: {submission_embeds_cp_array.device}")

Shape of test features (224309, 2)
shape: (5, 2)
┌──────────────────────┬─────────────────────────────────┐
│ protein_accession_id ┆ embedding_arrays                │
│ ---                  ┆ ---                             │
│ str                  ┆ array[f32, 480]                 │
╞══════════════════════╪═════════════════════════════════╡
│ Q8LX40               ┆ [-0.194824, -0.071106, … -0.33… │
│ Q8MHW5               ┆ [-0.164673, 0.078003, … -0.001… │
│ O00257               ┆ [-0.026779, -0.084534, … -0.02… │
│ A4FUD9               ┆ [-0.067444, -0.020065, … 0.046… │
│ Q9H1K1               ┆ [-0.268066, -0.078613, … -0.04… │
└──────────────────────┴─────────────────────────────────┘
prots_for_submission --  (224309,)
NumPy array shape: (224309, 480)
NumPy array dtype: float32
CuPy array shape: (224309, 480)
CuPy array dtype: float32
CuPy array device: <CUDA Device 0>


## Run the training pipeline

In [9]:
!pip install -U imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 4.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 82.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: sklearn-compat
    Found existing installation: sklearn-compat 0.1.3
    Uninstalling sklearn-compat-0.1.3:
      Successfully uninstalled sklearn-compat-0.1.3
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.13.0
    Uninstalling imbalanced-learn-0.13.0:
      Successfully uninstalled imbalanced-learn-0.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.8.0 

In [10]:
from imblearn.over_sampling import SMOTE

smote_sampler = SMOTE(sampling_strategy='auto', random_state=42)

ImportError: cannot import name 'UnsetMetadataPassedError' from 'sklearn.exceptions' (/usr/local/lib/python3.11/dist-packages/sklearn/exceptions.py)

In [11]:
def training_pipeline(features_path, labels_path, weights_path, training_iterations, c_value, proba_threshold, submission_filename):
    df_train_features = pl.read_parquet(features_path)
    print("Shape of training features", df_train_features.shape)
    print(df_train_features.head(5))

    df_train_labels = pl.read_parquet(labels_path)
    print("Shape of training labels", df_train_labels.shape)
    print(df_train_labels.head(5))

    label_cols = list(filter(lambda x: x != 'protein_accession_id', df_train_labels.columns))
    print("length of label_cols -- ", len(label_cols))

    df_weights = pl.read_csv(weights_path, separator="\t")
    df_weights.columns = ['go_term', 'ia']
    print("Shape of IA data", df_weights.shape)

    df_weights_filtered = df_weights.filter(pl.col('go_term').is_in(label_cols))
    print("shape of df_weights_filtered -- ", df_weights_filtered.shape)
    
    labels = df_weights_filtered.shape[0]
    class_wt_dict = {}
    
    for i in range(labels):
        class_wt_dict[df_weights_filtered.item(i,0)] = round(df_weights_filtered.item(i,1),5)

    embed_np_array = df_train_features['protein_embedding'].to_numpy().astype(np.float32)
    print(f"NumPy array shape: {embed_np_array.shape}")
    print(f"NumPy array dtype: {embed_np_array.dtype}")

    embeds_cp_array = cp.array(embed_np_array)
    print(f"CuPy array shape: {embeds_cp_array.shape}")
    print(f"CuPy array dtype: {embeds_cp_array.dtype}")
    print(f"CuPy array device: {embeds_cp_array.device}")

    go_terms_cp_array = cp.array(df_train_labels.select(label_cols).to_numpy())
    valid_rows_mask = (go_terms_cp_array != 0).any(axis=1)
    go_terms_cp_array_final = go_terms_cp_array[valid_rows_mask]
    print(f"CuPy array shape: {go_terms_cp_array_final.shape}")
    print(f"CuPy array dtype: {go_terms_cp_array_final.dtype}")
    print(f"CuPy array device: {go_terms_cp_array_final.device}")

    embeds_cp_array_final = embeds_cp_array[valid_rows_mask]
    print(f"CuPy array shape: {embeds_cp_array_final.shape}")
    print(f"CuPy array dtype: {embeds_cp_array_final.dtype}")
    print(f"CuPy array device: {embeds_cp_array_final.device}")

    scaler = StandardScaler()

    embeds_cp_array_final_scaled = scaler.fit_transform(embeds_cp_array_final)
    submission_embeds_cp_array_scaled = scaler.fit_transform(submission_embeds_cp_array)

    print("standard scaling of embeddings completed ....")

    label_weight_vector = cp.array([class_wt_dict.get(goterm, 0) for goterm in label_cols])

    # 2. Multiply the binary label matrix by the weight vector
    # This gives a 1D weight for every sample based on its positive GO terms
    sample_weights = go_terms_cp_array_final.dot(label_weight_vector)
    
    # 3. Normalize (Best practice: ensures the average weight is 1.0)
    sample_weights = sample_weights / np.mean(sample_weights)

    print("normalisation of weights completed .... ", sample_weights.shape)

    base_model = cuCML_LogisticRegression(solver='qn', max_iter=training_iterations, output_type='numpy', C=c_value, fit_intercept=False)

    multilabel_model = MultiOutputClassifier(base_model)
    
    print("Training model ...")
    multilabel_model.fit(embeds_cp_array_final_scaled.get(), go_terms_cp_array_final.get(), sample_weight=sample_weights.astype('float32'))
    print("Training complete ...")
    
    predictions_gpu = multilabel_model.predict_proba(submission_embeds_cp_array_scaled.get())
    prob_positive = np.transpose([p[:, 1] for p in predictions_gpu])
    print("predictions complete ... ", prob_positive.shape)

    predictions_df_pl = pl.DataFrame(
        prob_positive, # Ensure data is a numpy array
        schema=label_cols # Assign the column names
    )
    
    predictions_df_pl = predictions_df_pl.with_columns(
        pl.Series(name="protein_accession_id", values=prots_for_submission)
    )
    print("finished building the predictions_df_pl ... ", predictions_df_pl.shape)
    
    long_format_df = predictions_df_pl.melt(
        id_vars=["protein_accession_id"],          # Column to keep as identifier
        value_vars=label_cols,                  # Columns to melt into rows
        variable_name="go_term",                   # Name for the column containing GO terms
        value_name="probability"                   # Name for the column containing scores
    )
    long_format_df = long_format_df.filter(pl.col("probability") > proba_threshold)
    print("finished building the submission format ... ", long_format_df.shape)
    print(dd.sql("select count(distinct(protein_accession_id)), count(distinct(go_term)) from long_format_df").pl())
    
    long_format_df.write_csv(submission_filename, separator="\t", include_header=False)

In [36]:
weights_path = '/kaggle/input/cafa-6-protein-function-prediction/IA.tsv'
training_iterations = 2500
c_value = 0.01
proba_threshold = 0.75

file_configs = {
  "file1" : {
    "features_path" : "/kaggle/input/cafa6-protein-labels-features-depth-based-suman/train_protein_features_cc_2.parquet",
    "labels_path" : "/kaggle/input/cafa6-protein-labels-features-depth-based-suman/train_protein_labels_cc_2.parquet",
    "submission_filename" : "submission_df_cc_2.tsv"
  },
  "file2" : {
    "features_path" : "/kaggle/input/cafa6-protein-labels-features-depth-based-suman/train_protein_features_cc_3.parquet",
    "labels_path" : "/kaggle/input/cafa6-protein-labels-features-depth-based-suman/train_protein_labels_cc_3.parquet",
    "submission_filename" : "submission_df_cc_3.tsv"
  },
  "file3" : {
    "features_path" : "/kaggle/input/cafa6-protein-labels-features-depth-based-suman/train_protein_features_cc_4.parquet",
    "labels_path" : "/kaggle/input/cafa6-protein-labels-features-depth-based-suman/train_protein_labels_cc_4.parquet",
    "submission_filename" : "submission_df_cc_4.tsv"
  },
  "file4" : {
    "features_path" : "/kaggle/input/cafa6-protein-labels-features-depth-based-suman/train_protein_features_cc_5.parquet",
    "labels_path" : "/kaggle/input/cafa6-protein-labels-features-depth-based-suman/train_protein_labels_cc_5.parquet",
    "submission_filename" : "submission_df_cc_5.tsv"
  },
  "file5" : {
    "features_path" : "/kaggle/input/cafa6-protein-labels-features-depth-based-suman/train_protein_features_cc_6.parquet",
    "labels_path" : "/kaggle/input/cafa6-protein-labels-features-depth-based-suman/train_protein_labels_cc_6.parquet",
    "submission_filename" : "submission_df_cc_6.tsv"
  },
  "file6" : {
    "features_path" : "/kaggle/input/cafa6-protein-labels-features-depth-based-suman/train_protein_features_cc_concat.parquet",
    "labels_path" : "/kaggle/input/cafa6-protein-labels-features-depth-based-suman/train_protein_labels_cc_concat.parquet",
    "submission_filename" : "submission_df_cc_concat.tsv"
  }
}

In [37]:
for k,v in file_configs.items():
    print("k -> ",k)
    print("v -> ",v['features_path'])
    training_pipeline(v['features_path'], v['labels_path'] \
                  , weights_path, training_iterations, c_value, proba_threshold \
                  , v['submission_filename'])
    print(" ---- processing completed ---- ")

k ->  file1
v ->  /kaggle/input/cafa6-protein-labels-features-depth-based-suman/train_protein_features_cc_2.parquet
Shape of training features (33572, 2)
shape: (5, 2)
┌──────────────────────┬─────────────────────────────────┐
│ protein_accession_id ┆ protein_embedding               │
│ ---                  ┆ ---                             │
│ str                  ┆ array[f32, 480]                 │
╞══════════════════════╪═════════════════════════════════╡
│ P01186               ┆ [-0.191772, 0.081482, … 0.1107… │
│ Q13470               ┆ [-0.172485, -0.078552, … -0.05… │
│ A6NDV4               ┆ [-0.058075, 0.00692, … -0.0210… │
│ Q9H8V3               ┆ [-0.079163, -0.072571, … 0.029… │
│ Q5AMT2               ┆ [-0.19397, -0.09906, … 0.11492… │
└──────────────────────┴─────────────────────────────────┘
Shape of training labels (33572, 513)
shape: (5, 513)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ protein_a ┆ GO:000573 ┆ GO

## Combining all the submission datasets into one

In [38]:
all_dataframes = []
for dirpath, _, filenames in os.walk('/kaggle/working/'):
    for filename in filenames:
        # Construct the full file path by joining the directory path and the file name
        file_path = os.path.join(dirpath, filename) 
        if file_path.endswith('.tsv'):
            print(file_path)
            submission_df = pl.read_csv(file_path, separator="\t")
            submission_df.columns = ['protein','GO_Term','Probability']
            print("Shape - ", submission_df.shape)
            all_dataframes.append(submission_df)

/kaggle/working/submission_df_cc_5.tsv
Shape -  (236157, 3)
/kaggle/working/submission_df_mf_concat.tsv
Shape -  (84505, 3)
/kaggle/working/submission_df_bp_8.tsv
Shape -  (459418, 3)
/kaggle/working/submission_df_cc_3.tsv
Shape -  (661150, 3)
/kaggle/working/submission_df_bp_4.tsv
Shape -  (1870995, 3)
/kaggle/working/submission_df_mf_6.tsv
Shape -  (2683691, 3)
/kaggle/working/submission_df_bp_5.tsv
Shape -  (541804, 3)
/kaggle/working/submission_df_mf_5.tsv
Shape -  (540680, 3)
/kaggle/working/submission_df_bp_7.tsv
Shape -  (1398365, 3)
/kaggle/working/submission_df_mf_2.tsv
Shape -  (1276616, 3)
/kaggle/working/submission_df_mf_7.tsv
Shape -  (2152869, 3)
/kaggle/working/submission_df_cc_6.tsv
Shape -  (1232959, 3)
/kaggle/working/submission_df_bp_2.tsv
Shape -  (925256, 3)
/kaggle/working/submission_df_bp_6.tsv
Shape -  (2746238, 3)
/kaggle/working/submission_df_mf_3.tsv
Shape -  (464631, 3)
/kaggle/working/submission_df_bp_3.tsv
Shape -  (591253, 3)
/kaggle/working/submission_df

In [39]:
if all_dataframes:
    final_df = pl.concat(all_dataframes, how="vertical")
    print(final_df.shape)

final_df.write_csv("submission.tsv", separator="\t", include_header=False)

(22110790, 3)


In [40]:
print(dd.sql("select count(distinct(protein)) as proteins, \
count(distinct(GO_Term)) as GO_Terms, from final_df").pl())

shape: (1, 2)
┌──────────┬──────────┐
│ proteins ┆ GO_Terms │
│ ---      ┆ ---      │
│ i64      ┆ i64      │
╞══════════╪══════════╡
│ 137121   ┆ 8598     │
└──────────┴──────────┘


## Adding root terms

In [45]:
pattern = '*_cc_*.tsv'

all_dataframes_cc = []
for dirpath, _, filenames in os.walk('/kaggle/working/'):
    matching_files = fnmatch.filter(filenames, pattern)
    for filename in matching_files:
        # Construct the full file path by joining the directory path and the file name
        file_path = os.path.join(dirpath, filename) 
        print(file_path)
        submission_df = pl.read_csv(file_path, separator="\t")
        submission_df.columns = ['protein','GO_Term','Probability']
        print("Shape - ", submission_df.shape)
        all_dataframes_cc.append(submission_df)
        if all_dataframes_cc:
            final_df = pl.concat(all_dataframes_cc, how="vertical")
        
        unique_proteins_cc = final_df.select("protein").unique()

        new_rows_cc = unique_proteins_cc.with_columns(
            GO_Term = pl.lit("GO:0005575"),
            Probability = pl.lit(0.999999)
        )
        
        submission_df_cc_w_root = pl.concat([final_df, new_rows_cc])
        print(submission_df_cc_w_root.shape)

/kaggle/working/submission_df_cc_5.tsv
Shape -  (236157, 3)
(242269, 3)
/kaggle/working/submission_df_cc_3.tsv
Shape -  (661150, 3)
(908288, 3)
/kaggle/working/submission_df_cc_6.tsv
Shape -  (1232959, 3)
(2187114, 3)
/kaggle/working/submission_df_cc_4.tsv
Shape -  (1029097, 3)
(3221315, 3)
/kaggle/working/submission_df_cc_concat.tsv
Shape -  (32346, 3)
(3258291, 3)
/kaggle/working/submission_df_cc_2.tsv
Shape -  (195631, 3)
(3455177, 3)


In [46]:
pattern = '*_mf_*.tsv'

all_dataframes_mf = []
for dirpath, _, filenames in os.walk('/kaggle/working/'):
    matching_files = fnmatch.filter(filenames, pattern)
    for filename in matching_files:
        # Construct the full file path by joining the directory path and the file name
        file_path = os.path.join(dirpath, filename) 
        print(file_path)
        submission_df = pl.read_csv(file_path, separator="\t")
        submission_df.columns = ['protein','GO_Term','Probability']
        print("Shape - ", submission_df.shape)
        all_dataframes_mf.append(submission_df)
        if all_dataframes_mf:
            final_df = pl.concat(all_dataframes_mf, how="vertical")
        
        unique_proteins_mf = final_df.select("protein").unique()

        new_rows_mf = unique_proteins_mf.with_columns(
            GO_Term = pl.lit("GO:0003674"),
            Probability = pl.lit(0.999999)
        )
        
        submission_df_mf_w_root = pl.concat([final_df, new_rows_mf])
        print(submission_df_mf_w_root.shape)

/kaggle/working/submission_df_mf_concat.tsv
Shape -  (84505, 3)
(100970, 3)
/kaggle/working/submission_df_mf_6.tsv
Shape -  (2683691, 3)
(2792285, 3)
/kaggle/working/submission_df_mf_5.tsv
Shape -  (540680, 3)
(3335115, 3)
/kaggle/working/submission_df_mf_2.tsv
Shape -  (1276616, 3)
(4634213, 3)
/kaggle/working/submission_df_mf_7.tsv
Shape -  (2152869, 3)
(6801276, 3)
/kaggle/working/submission_df_mf_3.tsv
Shape -  (464631, 3)
(7268903, 3)
/kaggle/working/submission_df_mf_4.tsv
Shape -  (1418667, 3)
(8690040, 3)


In [47]:
pattern = '*_bp_*.tsv'

all_dataframes_bp = []
for dirpath, _, filenames in os.walk('/kaggle/working/'):
    matching_files = fnmatch.filter(filenames, pattern)
    for filename in matching_files:
        # Construct the full file path by joining the directory path and the file name
        file_path = os.path.join(dirpath, filename) 
        print(file_path)
        submission_df = pl.read_csv(file_path, separator="\t")
        submission_df.columns = ['protein','GO_Term','Probability']
        print("Shape - ", submission_df.shape)
        all_dataframes_bp.append(submission_df)
        if all_dataframes_bp:
            final_df = pl.concat(all_dataframes_bp, how="vertical")
        
        unique_proteins_bp = final_df.select("protein").unique()

        new_rows_bp = unique_proteins_bp.with_columns(
            GO_Term = pl.lit("GO:0008150"),
            Probability = pl.lit(0.999999)
        )
        
        submission_df_bp_w_root = pl.concat([final_df, new_rows_bp])
        print(submission_df_bp_w_root.shape)

/kaggle/working/submission_df_bp_8.tsv
Shape -  (459418, 3)
(493554, 3)
/kaggle/working/submission_df_bp_4.tsv
Shape -  (1870995, 3)
(2371216, 3)
/kaggle/working/submission_df_bp_5.tsv
Shape -  (541804, 3)
(2914420, 3)
/kaggle/working/submission_df_bp_7.tsv
Shape -  (1398365, 3)
(4328379, 3)
/kaggle/working/submission_df_bp_2.tsv
Shape -  (925256, 3)
(5269541, 3)
/kaggle/working/submission_df_bp_6.tsv
Shape -  (2746238, 3)
(8019634, 3)
/kaggle/working/submission_df_bp_3.tsv
Shape -  (591253, 3)
(8613505, 3)
/kaggle/working/submission_df_bp_concat.tsv
Shape -  (1568462, 3)
(10203528, 3)


In [48]:
submission_df = pl.concat([submission_df_mf_w_root, submission_df_cc_w_root, submission_df_bp_w_root])
print(submission_df.shape)

print(dd.sql("select count(distinct(protein)) as proteins, count(distinct(GO_Term)) as GO_Terms, from submission_df").pl())
submission_df.write_csv("submission.tsv", separator="\t", include_header=False)

(22348745, 3)
shape: (1, 2)
┌──────────┬──────────┐
│ proteins ┆ GO_Terms │
│ ---      ┆ ---      │
│ i64      ┆ i64      │
╞══════════╪══════════╡
│ 137121   ┆ 8601     │
└──────────┴──────────┘


## Miscellaneous operations

In [ ]:
terms_with_max_wts = (
    df_weights_filtered.top_k(35, by="ia")
    .get_column("go_term")
    .to_list()
)

In [ ]:
label_cols_subset = label_cols[0:5]
print(len(label_cols_subset))

In [ ]:
counts = cp.sum(go_terms_cp_array_final, axis=0)

# Transfer only the small summary back to CPU for printing
print("Positive counts per label:", counts.get().astype(int))

In [ ]:
empty_check = (go_terms_cp_array_final.sum(axis=0) == 0).any()
if empty_check:
    print("Warning: One of the 5 selected labels has no positive samples in this subset.")

In [ ]:
column_sums = np.sum(go_terms_cp_array_final, axis=1)

# Identify indices of empty labels (where sum is zero)
empty_indices = np.where(column_sums == 0)[0]
len(empty_indices)

In [ ]:
X_train_gpu, X_test_gpu, y_train_gpu, y_test_gpu = train_test_split(
    embeds_cp_array, go_terms_cp_array, test_size=0.1, random_state=42
)

print("\nShapes of train and test sets(GPU):")
print(f"X_train_gpu: {X_train_gpu.shape}")
print(f"X_test_gpu: {X_test_gpu.shape}")
print(f"y_train_gpu: {y_train_gpu.shape}")
print(f"y_test_gpu: {y_test_gpu.shape}")

In [ ]:
from sklearn.metrics import make_scorer, f1_score

param_grid = {
    'estimator__C': [0.1, 1.0, 10.0],           # Regularization strength
    'estimator__penalty': ['l2', 'none'],      # Penalty type
    'estimator__tol': [1e-4, 1e-3]             # Tolerance for stopping
}

# Create a scorer that suppresses the warning by setting f1 to 0.0 for zero-division cases
weighted_f1 = make_scorer(f1_score, average='weighted', zero_division=0)

# 3. Initialize GridSearchCV
# Use 'f1_macro' or 'f1_weighted' as GO terms are often highly imbalanced
grid_search = GridSearchCV(
    multilabel_model, 
    param_grid, 
    cv=5, 
    scoring=weighted_f1, 
    n_jobs=1
)

print("Training models (Grid Search)...")
# 4. Fit the grid search

grid_search.fit(embeds_cp_array_final.get(), go_terms_cp_array_final.get())

print("Training complete (Grid Search).")
# 5. Retrieve the best parameters
print("Optimal Hyperparameters:", grid_search.best_params_)

predictions_gpu = grid_search.predict_proba(submission_embeds_cp_array.get())

In [100]:
long_format_df.write_parquet("submission_df_cc_4.parquet")

In [96]:
import gzip

file_path = 'submission_df_cc_4.tsv.gz'

# Open the file using gzip.open in write-bytes mode ('wb')
with gzip.open(file_path, 'wb') as f:
    # Write the DataFrame to the file object
    long_format_df.write_csv(f, separator="\t", include_header=False)

OSError: 